In [2]:
from sympy import *
import numpy as np
from quaternion_filters import quaternion
from DCM_sym import HEB
from collections import OrderedDict
#init_printing()


Given that $q$ is a quaternion representation of the vehicles orientation and a vector is represented as a quaternion where the real part is zero and the i,j,k components are the vector (`quaternion.Quaternion.from_vec(some_vector)`).

To convert from inertial to body coordinate system:
$$
v_{body} = q^{-1} v_{inertial} q
$$
and to convert from body to inertial coordinate system:
$$
v_{inertial} = q v_{body} q^{-1}
$$

| State Var | Description            | Units   |
|:-----------|:------------------------|:---------:|
| $q_0$      | Quaternion    |  |
| $q_1$      | Quaternion   |  |
| $q_2$      | Quaternion     |  |
| $q_3$      | Quaternion     |  |
| $a_x$     | Body axis acceleration | m/s$^2$ |
| $a_y$     | Body axis acceleration | m/s$^2$ |
| $a_z$     | Body axis acceleration | m/s$^2$ |
| $w_x$     | Body axis rotation rate | rad/s |
| $w_y$     | Body axis rotation rate | rad/s |
| $w_z$     | Body axis rotation rate | rad/s |

In [3]:
q0, q1, q2, q3 = symbols(['q0', 'q1', 'q2', 'q3'])
wx, wy, wz = symbols(['wx', 'wy', 'wz'])
wbx, wby, wbz = symbols(['wbx', 'wby', 'wbz'])
abx, aby, abz = symbols(['abx', 'aby', 'abz'])
ax, ay, az = symbols(['ax', 'ay', 'az'])
mx, my, mz = symbols(['mx', 'my', 'mz'])
tas = symbols('tas')
state = [q0, q1, q2, q3, ax, ay, az, wx, wy, wz, wbx, wby, wbz, abx, aby, abz]

G = symbols('G')
dt = symbols('dt')

The model used for this filter is an assumption of coordinated flight.  Using this assumption, we can calculate the [rate of rotation](https://en.wikipedia.org/wiki/Standard_rate_turn) (about inertial z axis) and [centripital acceleration](https://en.wikipedia.org/wiki/Centripetal_force) due to this rotation.  

Centripital acceleration derivation by considering the triangle of a banked airplane with one leg of centripital acceleration and the other gravity.
$$ \tan \phi = \frac{a_c}{g} $$
where $\phi$ is the angle of bank.  $a_c=v^2/r$ so this equation can also be expressed as:
$$ \tan \phi = \frac{v^2}{r g} $$

Rate of rotation derivation. The aircraft completes a full rotation of a circle of radius $r$ in $2\pi r/v$ seconds.  Therefore, the rate of rotation (in inertial coordinates) can be written as:
$$ \dot{\psi} = \frac{2 \pi v}{2\pi r} = \frac{v}{r} = a_c/v $$
The rate of rotation simplifies to:
$$ \dot{\psi} = \frac{g}{v} \tan \phi $$

We will use the [Euler angles](https://en.wikipedia.org/wiki/Euler_angles#Tait%E2%80%93Bryan_angles) (more properly, they are the Tait-Bryan convention) for $\phi$ in the above equations as well as to create an intermediate coordinate system where the $y$ axis is the direction of the centripetal acceleration.

In [4]:
quaternion.arctan2 = atan2  # Make quaternion use symbolic functions
quaternion.arcsin = asin
quaternion.sqrt = sqrt
q = quaternion.Quaternion(q0,q1,q2,q3)
print("Roll from quaternion:", q.euler_angles()[0])
print("Heading from quaternion:", q.euler_angles()[2])

Roll from quaternion: atan2(2*q0*q1 + 2*q2*q3, -2*q1**2 - 2*q2**2 + 1)
Heading from quaternion: atan2(2*q0*q3 + 2*q1*q2, -2*q2**2 - 2*q3**2 + 1)


We now have enought information to start working through the Kalman filter update equations.

In [11]:
# Couple options here:
# 1) Propagate by current state rotations with very low Kalman Q components
# 2) Assume coordinated flight with higher Kalman Q components
qdelta = quaternion.Quaternion(1, wx/2*dt, wy/2*dt, wz/2*dt)  # small angle approximation of rotation
qnext = quaternion.Quaternion(q0,q1,q2,q3)*qdelta   # Option 1

phi = atan2(2*q0*q1 + 2*q2*q3, -2*q1**2 - 2*q2**2 + 1)  # Bank

# vector, in inertial coords, in direction which ac acts (along right wing (0,-1,0) for positive phi)
v_ac = (q*quaternion.Quaternion.from_vec([0,1.0,0])*q.inv()).as_ndarray()[1:]
v_ac[2] = 0
v_ac /= sqrt(v_ac[0]**2 + v_ac[1]**2)

ac = v_ac*G*tan(phi)
ag = np.array([0,0,-G])  
a = ag + ac

ab = (q.inv()*quaternion.Quaternion.from_vec(a)*q).as_ndarray()[1:]

wpredict = (q.inv()*quaternion.Quaternion.from_vec([0,0,G/tas*tan(phi)])*q).as_ndarray()[1:]
#wpredict = [wx, wy, wz]
# TODO: maybe this should be a combination of const w assumption and rate of turn
statenext = np.concatenate([qnext.as_ndarray(), ab, wpredict, [wbx, wby, wbz], [abx, aby, abz]])
statenext = np.array([x.subs(q0**2+q1**2+q2**2+q3**2,1) for x in statenext])
for n in range(len(state)):
    print(f"{state[n]}_next = {statenext[n]}")

q0_next = -dt*q1*wx/2 - dt*q2*wy/2 - dt*q3*wz/2 + q0
q1_next = dt*q0*wx/2 + dt*q2*wz/2 - dt*q3*wy/2 + q1
q2_next = dt*q0*wy/2 - dt*q1*wz/2 + dt*q3*wx/2 + q2
q3_next = dt*q0*wz/2 + dt*q1*wy/2 - dt*q2*wx/2 + q3
ax_next = q0*(0.5*G*q0*(2*q0*q1 + 2*q2*q3)*(-2.0*q0*q3 + 2.0*q1*q2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) + G*q2 + 0.5*G*q3*(2*q0*q1 + 2*q2*q3)*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1))) + q1*(0.5*G*q1*(2*q0*q1 + 2*q2*q3)*(-2.0*q0*q3 + 2.0*q1*q2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) + 0.5*G*q2*(2*q0*q1 + 2*q2*q3)*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) - G*q3) - q2*(-G*q0 - 0.5*G*q1*(2*q0*q1 + 2*q2*q3)*(1

In [12]:
# Check 60 degree bank to the right
# Should be 2g load factor
# Rate of turn should be: g/tas*tan(60) = g/tas*1.73
bank = 60*np.pi/180  
q60 = quaternion.Quaternion.axis_angle(np.array([0,0,1.0]), -np.pi/4)
q60 = q60*quaternion.Quaternion.axis_angle(np.array([0,1.0,0]), 0*np.pi/180)  # pitch up
q60 = q60*quaternion.Quaternion.axis_angle(np.array([1.0,0,0]), bank)
print("Euler angles (roll, pitch, heading):", q60.euler_angles()*180/np.pi)
q60 = q60.as_ndarray()
print("Accels:")
for n in range(4,7):
    tmp = statenext[n]
    for m in range(4):
        tmp = tmp.subs([q0,q1,q2,q3][m], q60[m])
    print(tmp)
    
print("Inertial rotation rates:")
q = quaternion.Quaternion(q0,q1,q2,q3)
w_inertial = (q*quaternion.Quaternion.from_vec(statenext[7:10])*q.inv()).as_ndarray()[1:]
for n in range(3):
    tmp = w_inertial[n]
    for m in range(4):
        tmp = tmp.subs([q0,q1,q2,q3][m], q60[m])
    print(tmp)
    
print("Body rotation rates:")
for n in range(3):
    tmp = wpredict[n]
    for m in range(4):
        tmp = tmp.subs([q0,q1,q2,q3][m], q60[m])
    print(tmp)


Euler angles (roll, pitch, heading): [60.0000000000000 0 -45.0000000000000]
Accels:
1.11022302462516e-16*G
0
-2.0*G
Inertial rotation rates:
0
-1.2490009027033e-16*G/tas
1.73205080756888*G/tas
Body rotation rates:
0
1.5*G/tas
0.866025403784439*G/tas


## Prediction Jacobian

$F$ is all the partial derivatives of the equations used to predict the next state.

In [13]:
F = np.zeros((len(state), len(state)), dtype=object)
for n in range(len(state)):
    for m in range(len(state)):
        tmp = diff(statenext[n], state[m])  
        #for k, v in subexpres.items():
        #    tmp = tmp.subs(v, symbols(k))
        #tmp = simplify(tmp)
        F[n, m] = tmp
        if tmp != 0:
            print(f"F[{n}, {m}] = {(tmp)}\n")

F[0, 0] = 1

F[0, 1] = -dt*wx/2

F[0, 2] = -dt*wy/2

F[0, 3] = -dt*wz/2

F[0, 7] = -dt*q1/2

F[0, 8] = -dt*q2/2

F[0, 9] = -dt*q3/2

F[1, 0] = dt*wx/2

F[1, 1] = 1

F[1, 2] = dt*wz/2

F[1, 3] = -dt*wy/2

F[1, 7] = dt*q0/2

F[1, 8] = -dt*q3/2

F[1, 9] = dt*q2/2

F[2, 0] = dt*wy/2

F[2, 1] = -dt*wz/2

F[2, 2] = 1

F[2, 3] = dt*wx/2

F[2, 7] = dt*q3/2

F[2, 8] = dt*q0/2

F[2, 9] = -dt*q1/2

F[3, 0] = dt*wz/2

F[3, 1] = dt*wy/2

F[3, 2] = -dt*wx/2

F[3, 3] = 1

F[3, 7] = -dt*q2/2

F[3, 8] = dt*q1/2

F[3, 9] = dt*q0/2

F[4, 0] = 0.5*G*q0*(2*q0*q1 + 2*q2*q3)*(-2.0*q0*q3 + 2.0*q1*q2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) + G*q2 + 0.5*G*q3*(2*q0*q1 + 2*q2*q3)*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) + q0*(1.0*G*q0*q1*(-2.0*q0*q3 + 2.0*q1*q2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2*

F[4, 3] = 0.5*G*q0*(2*q0*q1 + 2*q2*q3)*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) - G*q1 - 0.5*G*q3*(2*q0*q1 + 2*q2*q3)*(-2.0*q0*q3 + 2.0*q1*q2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) + q0*(-1.0*G*q0**2*(2*q0*q1 + 2*q2*q3)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) + 1.0*G*q0*q2*(-2.0*q0*q3 + 2.0*q1*q2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) + 0.5*G*q0*(2*q0*q1 + 2*q2*q3)*(-2.0*q0*q3 + 2.0*q1*q2)*(q0*(-q0*q3 + q1*q2) + 0.5*q3*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2))/(((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)**(3/2)*(-2*q1**2 - 2*q2**2 + 1)) + 1.0*G*q2*q3*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)/(sqrt((-q0*q3 + q

F[5, 2] = 0.5*G*q1*(2*q0*q1 + 2*q2*q3)*(-2.0*q0*q3 + 2.0*q1*q2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) + 0.5*G*q2*(2*q0*q1 + 2*q2*q3)*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) - G*q3 + q0*(1.0*G*q0*q2*(2*q0*q1 + 2*q2*q3)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) + 2.0*G*q0*q2*(2*q0*q1 + 2*q2*q3)*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)**2) + 1.0*G*q0*q3*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) + 0.5*G*q0*(2*q0*q1 + 2*q2*q3)*(-q1*(-q0*q3 + q1*q2) - 0.5*q2*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2))

F[6, 2] = 0.5*G*q0*(2*q0*q1 + 2*q2*q3)*(-2.0*q0*q3 + 2.0*q1*q2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) + G*q2 + 0.5*G*q3*(2*q0*q1 + 2*q2*q3)*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) + q0*(-2.0*G*q1*q2*(2*q0*q1 + 2*q2*q3)*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)**2) - 1.0*G*q1*q3*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)/(sqrt((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)*(-2*q1**2 - 2*q2**2 + 1)) - 0.5*G*q1*(2*q0*q1 + 2*q2*q3)*(-q1*(-q0*q3 + q1*q2) - 0.5*q2*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2))*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)/(((-q0*q3 + q1*q2)**2 + 0.25*(1.0*q0**2 - 1.0*q1**2 + 1.0*q2**2 - 1.0*q3**2)**2)**(3/2)*(-2*q1*

In [7]:
# Simplify with common expressions

subexprs, subs = cse(F.flatten())
subs = np.reshape(subs, (len(state), len(state)))

outputmode = 'ccode'  # 'ccode'

for se in subexprs:
    if outputmode == 'python':
        print(f"{se[0]} = {se[1]}")
    else:
        print(f"float {se[0]} = {ccode(se[1])};")
    
for n in range(len(state)):
    for m in range(len(state)):
        tmp = subs[n, m]
        if tmp != 0:
            if outputmode == 'python':
                print(f"F[{n}, {m}] = {tmp}")
            else:
                print(f"F({n}, {m}) = {ccode(tmp)};")

float x0 = (1.0/2.0)*dt;
float x1 = wx*x0;
float x2 = -x1;
float x3 = wy*x0;
float x4 = -x3;
float x5 = wz*x0;
float x6 = -x5;
float x7 = q1*x0;
float x8 = -x7;
float x9 = q2*x0;
float x10 = -x9;
float x11 = q3*x0;
float x12 = -x11;
float x13 = q0*x0;
float x14 = pow(q1, 2);
float x15 = 2*x14;
float x16 = pow(q2, 2);
float x17 = 2*x16;
float x18 = -x15 - x17 + 1;
float x19 = 1.0/x18;
float x20 = G*q3;
float x21 = x19*x20;
float x22 = pow(q0, 2);
float x23 = 1.0*x22;
float x24 = 1.0*x14;
float x25 = 1.0*x16;
float x26 = pow(q3, 2);
float x27 = 1.0*x26;
float x28 = x23 - x24 + x25 - x27;
float x29 = q1*q2;
float x30 = q0*q3;
float x31 = x29 - x30;
float x32 = 0.25*pow(x28, 2) + pow(x31, 2);
float x33 = pow(x32, -1.0/2.0);
float x34 = 1.0*x33;
float x35 = q1*x34;
float x36 = x28*x35;
float x37 = x21*x36;
float x38 = 2.0*x29 - 2.0*x30;
float x39 = G*q0;
float x40 = x19*x39;
float x41 = x38*x40;
float x42 = q0*q1;
float x43 = 2*q2*q3 + 2*x42;
float x44 = 0.5*x43;
float x45 = pow(x32, -3.0/2

## Accel update

The Kalman innovation is $y = z - h(x)$ and $H$ matrix is the partial derivatives of the $h(x)$ function.  Since accelerations are states in the filter, $h(x)$ is trivial.

In [8]:
haccel = np.array([ax + abx, ay + aby, az + abz])
H = np.zeros((3, len(state)), dtype=object)
for n in range(3):
    for m in range(len(state)):
        tmp = diff(haccel[n], state[m])
        H[n, m] = tmp
        if tmp != 0:
            print(f"H[{n}, {m}] = {tmp}")

H[0, 4] = 1
H[0, 13] = 1
H[1, 5] = 1
H[1, 14] = 1
H[2, 6] = 1
H[2, 15] = 1


## Gyro update

Similarly for the gyro, body rotation rates are part of the state so the $H$ matrix is trivial.

In [9]:
hgyro = np.array([wx + wbx, wy + wby, wz + wbz])
H = np.zeros((3, len(state)), dtype=object)
for n in range(3):
    for m in range(len(state)):
        tmp = diff(hgyro[n], state[m])
        H[n, m] = tmp
        if tmp != 0:
            print(f"H[{n}, {m}] = {tmp}")

H[0, 7] = 1
H[0, 10] = 1
H[1, 8] = 1
H[1, 11] = 1
H[2, 9] = 1
H[2, 12] = 1


## Magnetometer update

Unless the magnetometer is calibrated for hard and soft iron effects, I've found that the magnetometer does not provide reliable enough information for _orientation_.  For our purposes, we are interested in our orientation relative to magnetic north.  Therefore, a method that seems to work is to look at the body magnetometer vector in inertial coordinates in the inertial xy plane. This vector can be normalized, rotated back to body coordinates and used as the Kalman measurement.

 Only consider the normalized vector projected onto the global xy plane

1.  Rotate the magnetometer measurement to global coordinates
2.  Only look at component in xy plane and normalize.
3.  Rotate this back to body coordinates -- this is the "measurement"

Kalman equation:
$$y = z - h(x)$$
innovation = measurement - (func translating state to measurement)

What is $h(x)$? It is a function that converts the north vector into our measurement space (body coordinates).

In [10]:
q = quaternion.Quaternion(q0, q1, q2, q3)
roll, pitch, heading = q.euler_angles()

qtmp = 
mag_inertial = (q * quaternion.Quaternion.from_vec(np.array([mx, my, mz])) * q.inv()).as_ndarray()[1:]
mag_inertial[2] = 0
mag_inertial /= sqrt(mag_inertial[0]**2 + mag_inertial[1]**2)

mag_body = (q.inv() * quaternion.Quaternion.from_vec(mag_inertial) * q).as_ndarray()[1:]

# Only portion that we need to determine Jacobian is transfering "north vector"
# to body coordinates

h = np.array([cos(heading), sin(heading)])
H = np.zeros((2, len(state)), dtype=object)
for n in range(2):
    for m in range(len(state)):
        tmp = diff(h[n], state[m])
        tmp = tmp.subs(q0**2 + q1**2 + q2**2 + q3**2, 1.0)
        H[n, m] = tmp
        if tmp != 0:
            print(f"H[{n}, {m}] = {tmp}")


H[0, 0] = -2*q3*(2*q0*q3 + 2*q1*q2)*(-2*q2**2 - 2*q3**2 + 1)/((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2)**(3/2)
H[0, 1] = -2*q2*(2*q0*q3 + 2*q1*q2)*(-2*q2**2 - 2*q3**2 + 1)/((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2)**(3/2)
H[0, 2] = -4*q2/sqrt((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2) + (-2*q1*(2*q0*q3 + 2*q1*q2) + 4*q2*(-2*q2**2 - 2*q3**2 + 1))*(-2*q2**2 - 2*q3**2 + 1)/((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2)**(3/2)
H[0, 3] = -4*q3/sqrt((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2) + (-2*q0*(2*q0*q3 + 2*q1*q2) + 4*q3*(-2*q2**2 - 2*q3**2 + 1))*(-2*q2**2 - 2*q3**2 + 1)/((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2)**(3/2)
H[1, 0] = -2*q3*(2*q0*q3 + 2*q1*q2)**2/((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2)**(3/2) + 2*q3/sqrt((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2)
H[1, 1] = -2*q2*(2*q0*q3 + 2*q1*q2)**2/((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2)**(3/2) + 2*q2/sqrt((2*q0*q3 + 2*q1*q2)**2 + (

In [11]:
# Simplify with common expressions
subexprs, subs = cse(H.flatten())
subs = np.reshape(subs, (2, 10))

for se in subexprs:
    print(f"float {se[0]} = {ccode(se[1])};")
    
for n in range(2):
    for m in range(len(state)):
        tmp = subs[n, m]
        if tmp != 0:
            print(f"H({n}, {m}) = {(tmp)};")  # ccode

float x0 = 2*q3;
float x1 = 2*q0;
float x2 = 2*q1;
float x3 = q2*x2 + q3*x1;
float x4 = -2*pow(q2, 2) - 2*pow(q3, 2) + 1;
float x5 = pow(x3, 2);
float x6 = pow(x4, 2) + x5;
float x7 = pow(x6, -3.0/2.0);
float x8 = x4*x7;
float x9 = x3*x8;
float x10 = 2*q2;
float x11 = pow(x6, -1.0/2.0);
float x12 = 4*q2;
float x13 = x12*x4 - x2*x3;
float x14 = 4*q3;
float x15 = -x1*x3 + x14*x4;
float x16 = 2*x11;
float x17 = x5*x7;
float x18 = x3*x7;
H(0, 0) = -x0*x9;
H(0, 1) = -x10*x9;
H(0, 2) = -x11*x12 + x13*x8;
H(0, 3) = -x11*x14 + x15*x8;
H(1, 0) = q3*x16 - x0*x17;
H(1, 1) = q2*x16 - x10*x17;
H(1, 2) = x11*x2 + x13*x18;
H(1, 3) = x1*x11 + x15*x18;


In [35]:
# What if measurement is heading euler angle?
q = quaternion.Quaternion(q0, q1, q2, q3)
heading = atan2(2*q0*q3 + 2*q1*q2, -2*q2**2 - 2*q3**2 + 1)
mag_inertial = (q * quaternion.Quaternion.from_vec(np.array([mx, my, mz])) * q.inv()).as_ndarray()[1:]
mag_inertial[2] = 0
mag_inertial /= sqrt(mag_inertial[0]**2 + mag_inertial[1]**2)

#sensor_heading = np.array([mx, my])
#sensor_heading /= np.linalg.norm(sensor_heading)

h = np.vstack([cos(heading), sin(heading)])

#y = np.vstack(sensor_heading) - h

H = np.zeros((2, len(state)), dtype=object)
for n in range(2):
    for m in range(len(state)):
        tmp = diff(h[n, 0], state[m])
        #tmp = tmp.subs(q0**2 + q1**2 + q2**2 + q3**2, 1.0)
        H[n, m] = tmp
        if tmp != 0:
            print(f"H[{n}, {m}] = {tmp}")

H[0, 0] = -2*q3*(2*q0*q3 + 2*q1*q2)*(-2*q2**2 - 2*q3**2 + 1)/((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2)**(3/2)
H[0, 1] = -2*q2*(2*q0*q3 + 2*q1*q2)*(-2*q2**2 - 2*q3**2 + 1)/((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2)**(3/2)
H[0, 2] = -4*q2/sqrt((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2) + (-2*q1*(2*q0*q3 + 2*q1*q2) + 4*q2*(-2*q2**2 - 2*q3**2 + 1))*(-2*q2**2 - 2*q3**2 + 1)/((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2)**(3/2)
H[0, 3] = -4*q3/sqrt((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2) + (-2*q0*(2*q0*q3 + 2*q1*q2) + 4*q3*(-2*q2**2 - 2*q3**2 + 1))*(-2*q2**2 - 2*q3**2 + 1)/((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2)**(3/2)
H[1, 0] = -2*q3*(2*q0*q3 + 2*q1*q2)**2/((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2)**(3/2) + 2*q3/sqrt((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2)
H[1, 1] = -2*q2*(2*q0*q3 + 2*q1*q2)**2/((2*q0*q3 + 2*q1*q2)**2 + (-2*q2**2 - 2*q3**2 + 1)**2)**(3/2) + 2*q2/sqrt((2*q0*q3 + 2*q1*q2)**2 + (

## GPS Bearing Update

y = meas - h(x)



In [30]:
# Recall that heading from quaternion is:
# Heading from quaternion: atan2(2*q0*q3 + 2*q1*q2, -2*q2**2 - 2*q3**2 + 1)
v_head = np.array([-2*q2**2 - 2*q3**2 + 1, 2*q0*q3 + 2*q1*q2])

# v_bearing = np.array([np.cos(bearing), np.sin(bearing)])
    
for n in range(2):
    for m in range(len(state)):
        tmp = diff(v_head[n], state[m])
        tmp = tmp.subs(q0**2 + q1**2 + q2**2 + q3**2, 1.0)
        if tmp != 0:
            print(f"H[{n}, {m}] = {tmp}")

H[0, 2] = -4*q2
H[0, 3] = -4*q3
H[1, 0] = 2*q3
H[1, 1] = 2*q2
H[1, 2] = 2*q1
H[1, 3] = 2*q0


In [115]:
# Given euler angle uncertainties, calculate q uncertainties
quaternion.sin = sin
quaternion.cos = cos
roll, pitch, heading = symbols(['roll', 'pitch', 'heading'])

q = quaternion.Quaternion.axis_angle(np.array([0, 0, 1.0]), heading)
q = q*quaternion.Quaternion.axis_angle(np.array([0, 1.0, 0]), pitch)
q = q*quaternion.Quaternion.axis_angle(np.array([1.0, 0, 0]), roll)
q = q.as_ndarray()
J = np.zeros((4, 3), dtype=object)
for m in range(4):
    for n in range(3):
        J[m, n] = diff(q[m], [roll, pitch, heading][n]).subs(roll, 0).subs(pitch, 5*pi/180).subs(heading, pi)
        if J[m, n] != 0:
            print(f"J[{m}, {n}] = {J[m,n]}")
            
heading = 180*np.pi/180
pitch = 0
roll = 0

print(J.dot(np.array([0, 0, (5*np.pi/180)**2])))

AttributeError: 'Mul' object has no attribute 'eval'